In [12]:
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime
import json
import queue
import threading

import fundamentalanalysis as fa
import fa_mods as famods
import fa_utils

with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [2]:
all_companies = fa.available_companies(api_key)
tradeable_companies = famods.tradable_companies(api_key)

In [3]:
exchanges = all_companies.groupby('exchangeShortName')['exchange'].agg('unique').reset_index().rename(columns={'exchangeShortName': 'short_name', 'unique': 'exhange'})

In [4]:
datestamp = datetime.now().strftime('%y%m%d')

with open(f'../data/all_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(all_companies, file)

with open(f'../data/all_tradable_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(tradeable_companies, file)

with open(f'../data/exchanges_{datestamp}.pkl', 'wb') as file:
    pickle.dump(exchanges, file)

In [5]:
lse_companies = all_companies[(all_companies.exchangeShortName == 'LSE')]
nyse_companies = all_companies[(all_companies.exchangeShortName == 'NYSE')]
nasdaq_companies = all_companies[(all_companies.exchangeShortName == 'NASDAQ')]

lse_stocks = lse_companies[(lse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nyse_stocks = nyse_companies[(nyse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nasdaq_stocks = nasdaq_companies[(nasdaq_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

print(f"num LSE companies: {len(lse_companies)}")
print(f"num LSE stocks: {len(lse_stocks)}")
print()
print(f"num NYSE companies: {len(nyse_companies)}")
print(f"num NYSE stocks: {len(nyse_stocks)}")
print()
print(f"num NASDAQ companies: {len(nasdaq_companies)}")
print(f"num NASDAQ stocks: {len(nasdaq_stocks)}")


num LSE companies: 6335
num LSE stocks: 3877

num NYSE companies: 4060
num NYSE stocks: 3463

num NASDAQ companies: 10219
num NASDAQ stocks: 5916


In [43]:
nyse_stock_data = {}
failed = []

symbol_queue = queue.Queue()
[symbol_queue.put(s) for s in nyse_stocks.symbol.tolist()]
symbol_queue.qsize()

def worker():
    while True:
        symbol = symbol_queue.get()
        print(f'Working on {symbol}')

        if symbol in nyse_stock_data:
            print(f"Already have {symbol}, skipping")
            continue

        try:
            data = fa_utils.get_ticker_data(symbol, api_key)
            nyse_stock_data[symbol] = data
            print(f"{symbol} SUCCESS")
        except:
            print(f"{symbol} FAILED")
            failed.append(symbol)

        symbol_queue.task_done()

for x in range(4):
    name = "Thread_" + str(x)
    t = threading.Thread(name=name,target=worker, daemon=True).start()

symbol_queue.join()

Working on PBC
Working on ATEST-A
Working on UPH-WT
Working on LNN
ATEST-A FAILED
Working on WAL-PA
UPH-WT FAILED
Working on BLND
PBC SUCCESS
Working on LOKM-WT
WAL-PA SUCCESS
Working on WRB-PE
LNN SUCCESS
Working on CAL
BLND SUCCESS
Working on TAC
LOKM-WT FAILED
Working on JOBY-WT
WRB-PE FAILED
Working on CNO
CAL FAILED
Working on GLT
JOBY-WT FAILED
Working on WMB
TAC SUCCESS
Working on HOUS
Finished BAC-PK
Working on HUGS-WT
HUGS-WT FAILED
Finished HUGS-WT
Working on JWSM-WT
GLP-PA FAILED
Finished GLP-PA
Working on SHPW-WT
CNO SUCCESS
Working on CM
SHPW-WT FAILED
Finished SHPW-WT
Working on CPA
GLT SUCCESS
Working on ZBH
WMB SUCCESS
Working on TDCX
RH FAILED
Finished RH
Working on DNMR
TDCX FAILED
Working on WFC-PD
JWSM-WT FAILED
Finished JWSM-WT
Working on PSF
HOUS SUCCESS
Working on GSQD-WT
PSF FAILED
Finished PSF
Working on GIC
GSQD-WT FAILED
Working on EDR
Finished CPA
Working on SHG
CM SUCCESS
Working on DIS
ZBH SUCCESS
Working on CUBI-PE
Finished DNMR
Working on VMW
WFC-PD SUCC

Exception in thread Thread_2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_809894/1211652654.py", line 25, in worker
  File "/usr/lib/python3.10/queue.py", line 75, in task_done
    raise ValueError('task_done() called too many times')
ValueError: task_done() called too many times


JEMD SUCCESS


Exception in thread Thread_0:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_809894/1211652654.py", line 25, in worker
  File "/usr/lib/python3.10/queue.py", line 75, in task_done
    raise ValueError('task_done() called too many times')
ValueError: task_done() called too many times


NMS SUCCESS


Exception in thread Thread_1:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_809894/3758640301.py", line 24, in worker
  File "/usr/lib/python3.10/queue.py", line 75, in task_done
    raise ValueError('task_done() called too many times')
ValueError: task_done() called too many times


Finished CHPT


In [44]:
print(len(failed))
print(len(nyse_stock_data))

938
2528


In [45]:
with open(f'../data/all_nyse_stock_data_{datestamp}.pkl', 'wb') as file:
    pickle.dump(nyse_stock_data, file)

with open(f'../data/all_nyse_stock_data_{datestamp}_failed.pkl', 'wb') as file:
    pickle.dump(failed, file)

In [47]:
with open(f'../data/all_nyse_stock_data_{datestamp}.pkl', 'rb') as file:
    nyse_stock_data_2 = pickle.load(file)

len(nyse_stock_data_2)

2528

In [51]:
nyse_stock_data['DIS']

{'timestamp': 1671573667.03506,
 'income_statement':                                                         2022  \
 reportedCurrency                                         USD   
 cik                                               0001744489   
 fillingDate                                       2022-11-29   
 acceptedDate                             2022-11-29 17:03:15   
 revenue                                          82722000000   
 costOfRevenue                                    54401000000   
 grossProfit                                      28321000000   
 grossProfitRatio                                    0.342364   
 researchAndDevelopmentExpenses                           0.0   
 generalAndAdministrativeExpenses                         0.0   
 sellingAndMarketingExpenses                              0.0   
 sellingGeneralAndAdministrativeExpenses          16388000000   
 otherExpenses                                     5163000000   
 operatingExpenses                    